In [1]:
#Online kernal learning for active sensor network 
# This code demonstrates how to integrate online learning with a simple active learning strategy. We'll simulate a basic sensor  
# data stream and implement a kernelized learning model using the scikit-learn library.
!pip install numpy scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# OnlineKernelLearning Class: This class implements online learning using a kernel approximation (RBF kernel) and stochastic 
# gradient descent (SGD) for classification. It supports both fitting and prediction.


In [3]:
class OnlineKernelLearning:
    def __init__(self, n_features, gamma=1.0):
        self.rbf_feature = RBFSampler(gamma=gamma, n_components=n_features)
        self.base_model = SGDClassifier(loss='hinge', warm_start=True)
        self.model = CalibratedClassifierCV(self.base_model)  
        self.is_fitted = False
        self.X_accumulated = []
        self.y_accumulated = []
        self.classes_ = None 

    def fit(self, X, y):
        self.X_accumulated.extend(X)
        self.y_accumulated.extend(y)
        X_transformed = self.rbf_feature.fit_transform(self.X_accumulated)
        
        if not self.is_fitted:
            self.classes_ = np.unique(y)
            self.base_model.partial_fit(X_transformed, self.y_accumulated, classes=self.classes_)  # Call partial_fit on base model first
            self.model.fit(X_transformed, self.y_accumulated)  # Then fit the calibrated model
            self.is_fitted = True
        else:
            self.base_model.partial_fit(X_transformed, self.y_accumulated)  # Subsequent calls to partial_fit on base model
            self.model = CalibratedClassifierCV(self.base_model) # Re-calibrate the model with the updated base model
            self.model.fit(X_transformed, self.y_accumulated)  # Re-fit the calibrated model
    def predict(self, X):
        X_transformed = self.rbf_feature.transform(X)
        return self.model.predict(X_transformed)

def simulate_sensor_data(n_samples):
    # Simulating some sensor data
    X = np.random.rand(n_samples, 2)  # 2 features
    y = (X[:, 0] + X[:, 1] > 1).astype(int)  # Simple decision boundary
    return X, y
# def predict_proba(self, X):
#         X_transformed = self.rbf_feature.transform(X)
#         return self.model.predict_proba(X_transformed)
# 

#
def uncertainty_sampling(model, X, n_samples=5):
    # Transform input data before prediction
    X_transformed = model.rbf_feature.transform(X) 
    probabilities = model.model.predict_proba(X_transformed)
    uncertainty = 1 - np.max(probabilities, axis=1)
    uncertain_indices = np.argsort(uncertainty)[-n_samples:]
    return uncertain_indices

## Main Simulation Loop:
# Initial data is generated and used to fit the model.
# New sensor data is simulated and processed one sample at a time.
# Every few samples, the model checks for uncertainty and prints uncertain indices.
# Main Simulation Loop
n_initial_samples = 20
n_new_samples = 50
n_features = 100

# Initialize online kernel learner
online_model = OnlineKernelLearning(n_features=n_features)

# Simulate initial data
X_initial, y_initial = simulate_sensor_data(n_initial_samples)
online_model.fit(X_initial, y_initial)
# Simulating Sensor Data: The simulate_sensor_data function generates random data for demonstration purposes. 
# The labels are assigned based on a simple decision boundary.
# Simulate new incoming sensor data
X_new, y_new = simulate_sensor_data(n_new_samples)

# for i in range(n_new_samples):
#     # Fit model with new data
#     online_model.fit(X_new[i:i+1], y_new[i:i+1])
    
# Uncertainty Sampling: The uncertainty_sampling function selects samples that the model 
# is most uncertain about, which can be queried for labels.
for i in range(n_new_samples):
    # ...

    # Active learning: choose next sample based on uncertainty
    if (i + 1) % 5 == 0:  # Every 5 samples
        # Check if there are enough samples for uncertainty sampling
        remaining_samples = len(X_new) - (i + 1)  
        if remaining_samples > 0:
            # Use min to ensure we don't try to select more samples than available
            num_samples_to_select = min(remaining_samples, 5) 
            uncertain_indices = uncertainty_sampling(online_model, X_new[i+1:i+1+num_samples_to_select])
            print(f"Uncertain indices: {uncertain_indices}")


#Accuracy Evaluation: At the end of the process, the accuracy of the model on the new data is printed
# Evaluation
y_pred = online_model.predict(X_new)
accuracy = accuracy_score(y_new, y_pred)
print(f"Accuracy on new data: {accuracy:.2f}")

Uncertain indices: [3 4 0 1 2]
Uncertain indices: [4 1 2 3 0]
Uncertain indices: [2 3 4 1 0]
Uncertain indices: [1 0 4 3 2]
Uncertain indices: [0 4 1 2 3]
Uncertain indices: [0 4 2 1 3]
Uncertain indices: [0 3 1 4 2]
Uncertain indices: [4 2 0 3 1]
Uncertain indices: [3 4 2 0 1]
Accuracy on new data: 0.92


In [ ]:

#python active_sensor_network.py
